# AutoML - Classification

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup_cls)  
* [Compare Models](#compare_cls)  
* [Create Model](#create_cls)  
* [Tune Model](#tune_cls)  
* [Evaluate Model](#evaluate_cls)  
* [Finalize and Store Model](#finalize_and_store_cls)

## Imports and Global Settings

In [1]:
import os
import sys
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.metrics import accuracy_score, precision_score
from pycaret.classification import ClassificationExperiment

here = os.getcwd()
sys.path.append(os.path.join(here, ".."))

from src.utils.modeling_utils import ModelSetup, calculate_roi, save_model_report

load_dotenv()
RDS_ENDPOINT = os.getenv("RDS_ENDPOINT")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

## Load Data

In [2]:
username = "postgres"
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = "nba_betting"
port = "5432"

# Create the connection string
connection_string = (
    f"postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}"
)

### Games

In [3]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = [
    "game_id",
    "game_datetime",
    "home_team",
    "away_team",
    "open_line",
    "home_score",
    "away_score",
    "game_completed",
    "odds_last_update",
    "scores_last_update",
]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
games_query = f"SELECT {', '.join(features)} FROM games WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    games = pd.read_sql_query(games_query, connection)

### Features

In [4]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = ["game_id", "data"]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
features_query = f"SELECT {', '.join(features)} FROM all_features_json WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    all_features = pd.read_sql_query(features_query, connection)

# Normalize the JSON strings in the 'data' column
expanded_data = pd.json_normalize(all_features["data"])

# Drop the original 'data' column and concatenate the expanded data
all_features = pd.concat([all_features.drop(columns=["data"]), expanded_data], axis=1)

In [5]:
games_features = pd.merge(
    games,
    all_features,
    on="game_id",
    how="left",
    validate="one_to_one",
    suffixes=("", "_drop"),
)
# Drop the columns from df2 (with suffix '_drop')
games_features = games_features[
    games_features.columns.drop(list(games_features.filter(regex="_drop")))
]

<a id='basic_data_overview'></a>

## Basic Data Overview

In [6]:
df = games_features.copy()

In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3956 entries, 0 to 3955
Data columns (total 778 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3956 non-null   object        
 1    game_datetime                                 3956 non-null   datetime64[ns]
 2    home_team                                     3956 non-null   object        
 3    away_team                                     3956 non-null   object        
 4    open_line                                     3853 non-null   float64       
 5    home_score                                    3855 non-null   float64       
 6    away_score                                    3855 non-null   float64       
 7    game_completed                                3956 non-null   bool          
 8    odds_last_update                              95 non-nul

In [8]:
df.head(10)

,game_id,game_datetime,home_team,away_team,open_line,home_score,away_score,game_completed,odds_last_update,scores_last_update,season,away_ATL,away_BKN,away_BOS,away_CHA,away_CHI,away_CLE,away_DAL,away_DEN,away_DET,away_GSW,away_HOU,away_IND,away_LAC,away_LAL,away_MEM,away_MIA,away_MIL,away_MIN,away_NOP,away_NYK,away_OKC,away_ORL,away_PHI,away_PHX,away_POR,away_SAC,away_SAS,away_TOR,away_UTA,away_WAS,elo1_pre,elo2_pre,home_ATL,home_BKN,home_BOS,home_CHA,home_CHI,home_CLE,home_DAL,home_DEN,home_DET,home_GSW,home_HOU,home_IND,home_LAC,home_LAL,home_MEM,home_MIA,home_MIL,home_MIN,home_NOP,home_NYK,home_OKC,home_ORL,home_PHI,home_PHX,home_POR,home_SAC,home_SAS,home_TOR,home_UTA,home_WAS,538_prob1,elo_prob1,elo_prob2,last_5_hv,streak_hv,win_pct_hv,raptor1_pre,raptor2_pre,season_type,raptor_prob1,raptor_prob2,rest_diff_hv,carm_elo1_pre,carm_elo2_pre,day_of_season,point_diff_hv,carm_elo_prob1,carm_elo_prob2,away_team_last_5,away_team_streak,home_team_last_5,home_team_streak,away_team_win_pct,home_team_win_pct,point_diff_last_5_hv,pie_away_all_advanced,pie_away_l2w_advanced,pie_home_all_advanced,pie_home_l2w_advanced,l_away_all_traditional,l_away_l2w_traditional,l_home_all_traditional,l_home_l2w_traditional,pace_away_all_advanced,pace_away_l2w_advanced,pace_home_all_advanced,pace_home_l2w_advanced,poss_away_all_advanced,poss_away_l2w_advanced,poss_home_all_advanced,poss_home_l2w_advanced,w_away_all_traditional,w_away_l2w_traditional,w_home_all_traditional,w_home_l2w_traditional,gp_away_all_traditional,gp_away_l2w_traditional,gp_home_all_traditional,gp_home_l2w_traditional,ast_to_away_all_advanced,ast_to_away_l2w_advanced,ast_to_home_all_advanced,ast_to_home_l2w_advanced,away_team_avg_point_diff,e_pace_away_all_advanced,e_pace_away_l2w_advanced,e_pace_home_all_advanced,e_pace_home_l2w_advanced,home_team_avg_point_diff,min_away_all_traditional,min_away_l2w_traditional,min_home_all_traditional,min_home_l2w_traditional,opp_pf_away_all_opponent,opp_pf_away_l2w_opponent,opp_pf_home_all_opponent,opp_pf_home_l2w_opponent,pts_away_all_traditional,pts_away_l2w_traditional,pts_home_all_traditional,pts_home_l2w_traditional,ts_pct_away_all_advanced,ts_pct_away_l2w_advanced,ts_pct_home_all_advanced,ts_pct_home_l2w_advanced,ast_pct_away_all_advanced,ast_pct_away_l2w_advanced,ast_pct_home_all_advanced,ast_pct_home_l2w_advanced,days_since_last_game_away,days_since_last_game_home,efg_pct_away_all_advanced,efg_pct_away_l2w_advanced,efg_pct_home_all_advanced,efg_pct_home_l2w_advanced,opp_ast_away_all_opponent,opp_ast_away_l2w_opponent,opp_ast_home_all_opponent,opp_ast_home_l2w_opponent,opp_blk_away_all_opponent,opp_blk_away_l2w_opponent,opp_blk_home_all_opponent,opp_blk_home_l2w_opponent,opp_fga_away_all_opponent,opp_fga_away_l2w_opponent,opp_fga_home_all_opponent,opp_fga_home_l2w_opponent,opp_fgm_away_all_opponent,opp_fgm_away_l2w_opponent,opp_fgm_home_all_opponent,opp_fgm_home_l2w_opponent,opp_fta_away_all_opponent,opp_fta_away_l2w_opponent,opp_fta_home_all_opponent,opp_fta_home_l2w_opponent,opp_ftm_away_all_opponent,opp_ftm_away_l2w_opponent,opp_ftm_home_all_opponent,opp_ftm_home_l2w_opponent,opp_pfd_away_all_opponent,opp_pfd_away_l2w_opponent,opp_pfd_home_all_opponent,opp_pfd_home_l2w_opponent,opp_pts_away_all_opponent,opp_pts_away_l2w_opponent,opp_pts_home_all_opponent,opp_pts_home_l2w_opponent,opp_reb_away_all_opponent,opp_reb_away_l2w_opponent,opp_reb_home_all_opponent,opp_reb_home_l2w_opponent,opp_stl_away_all_opponent,opp_stl_away_l2w_opponent,opp_stl_home_all_opponent,opp_stl_home_l2w_opponent,opp_tov_away_all_opponent,opp_tov_away_l2w_opponent,opp_tov_home_all_opponent,opp_tov_home_l2w_opponent,reb_pct_away_all_advanced,reb_pct_away_l2w_advanced,reb_pct_home_all_advanced,reb_pct_home_l2w_advanced,dreb_pct_away_all_advanced,dreb_pct_away_l2w_advanced,dreb_pct_home_all_advanced,dreb_pct_home_l2w_advanced,opp_blka_away_all_opponent,opp_blka_away_l2w_opponent,opp_blka_home_all_opponent,opp_blka_home_l2w_opponent,opp_dreb_away_all_opponent,opp_dreb_

## Data Preparation

#### Drop Non-Completed Games and Games with No Line

In [9]:
df = df[df["game_completed"] == True]
df = df.dropna(subset=["open_line"])

In [10]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3853 entries, 31 to 3955
Data columns (total 778 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3853 non-null   object        
 1    game_datetime                                 3853 non-null   datetime64[ns]
 2    home_team                                     3853 non-null   object        
 3    away_team                                     3853 non-null   object        
 4    open_line                                     3853 non-null   float64       
 5    home_score                                    3853 non-null   float64       
 6    away_score                                    3853 non-null   float64       
 7    game_completed                                3853 non-null   bool          
 8    odds_last_update                              0 non-nul

### Create Targets

In [11]:
df = ModelSetup.add_targets(df)

### Select Features

In [12]:
# training_seasons = [x for x in range(2010, 2022)]
training_dates, testing_dates = ModelSetup.choose_dates([2020, 2021], [2022], "Reg")
print("Training Dates:")
print(training_dates)
print("Testing Dates:")
print(testing_dates)

Training Dates:
('2020-12-22', '2022-04-10')
Testing Dates:
('2022-10-18', '2023-04-09')


In [13]:
for col in df.columns:
    print(col)

game_id
game_datetime
home_team
away_team
open_line
home_score
away_score
game_completed
odds_last_update
scores_last_update
season
away_ATL
away_BKN
away_BOS
away_CHA
away_CHI
away_CLE
away_DAL
away_DEN
away_DET
away_GSW
away_HOU
away_IND
away_LAC
away_LAL
away_MEM
away_MIA
away_MIL
away_MIN
away_NOP
away_NYK
away_OKC
away_ORL
away_PHI
away_PHX
away_POR
away_SAC
away_SAS
away_TOR
away_UTA
away_WAS
elo1_pre
elo2_pre
home_ATL
home_BKN
home_BOS
home_CHA
home_CHI
home_CLE
home_DAL
home_DEN
home_DET
home_GSW
home_HOU
home_IND
home_LAC
home_LAL
home_MEM
home_MIA
home_MIL
home_MIN
home_NOP
home_NYK
home_OKC
home_ORL
home_PHI
home_PHX
home_POR
home_SAC
home_SAS
home_TOR
home_UTA
home_WAS
538_prob1
elo_prob1
elo_prob2
last_5_hv
streak_hv
win_pct_hv
raptor1_pre
raptor2_pre
season_type
raptor_prob1
raptor_prob2
rest_diff_hv
carm_elo1_pre
carm_elo2_pre
day_of_season
point_diff_hv
carm_elo_prob1
carm_elo_prob2
away_team_last_5
away_team_streak
home_team_last_5
home_team_streak
away_team_win_pct
ho

In [14]:
features_to_use = [
    "open_line",
    "rest_diff_hv",
    "day_of_season",
    "last_5_hv",
    "538_prob1",
    "elo_prob1",
    "streak_hv",
    "point_diff_last_5_hv",
    "point_diff_hv",
    "win_pct_hv",
    "plus_minus_home_l2w_traditional",
    "net_rating_home_l2w_advanced",
    "plus_minus_home_l2w_opponent",
    "plus_minus_zscore_home_l2w_traditional",
    "net_rating_zscore_home_l2w_advanced",
    "plus_minus_zscore_home_l2w_opponent",
    "e_net_rating_home_l2w_advanced",
    "e_net_rating_zscore_home_l2w_advanced",
    "plus_minus_percentile_home_l2w_opponent",
    "plus_minus_percentile_home_l2w_traditional",
    "net_rating_percentile_home_l2w_advanced",
    "plus_minus_away_l2w_traditional",
    "plus_minus_away_l2w_opponent",
    "w_pct_zscore_home_l2w_traditional",
    "e_net_rating_percentile_home_l2w_advanced",
    "e_net_rating_away_l2w_advanced",
    "pie_percentile_home_l2w_advanced",
    "e_net_rating_zscore_away_l2w_advanced",
    "net_rating_zscore_away_l2w_advanced",
    "pie_home_l2w_advanced",
]

In [15]:
df.dropna(subset=features_to_use, inplace=True)

In [16]:
training_df, testing_df, model_report = ModelSetup.create_datasets(
    df, "cls", features_to_use, training_dates, testing_dates, create_report=True
)

In [17]:
print("Training Shape: ", training_df.shape)
print("Testing Shape: ", testing_df.shape)

Training Shape:  (2330, 32)
Testing Shape:  (1199, 32)


In [ ]:
training_df.info(verbose=True, null_counts=True)

### Baselines

In [ ]:
training_baseline_home_team = model_report["ind_baseline_train"]
testing_baseline_home_team = model_report["ind_baseline_test"]

training_baseline_majority_class = model_report["dep_baseline_train"]
testing_baseline_majority_class = model_report["dep_baseline_test"]

print(f"Training Baseline - Home Team: {training_baseline_home_team:.2f}")
print(f"Testing Baseline - Home Team: {testing_baseline_home_team:.2f}")
print(f"Training Baseline - Majority Class: {training_baseline_majority_class:.2f}")
print(f"Testing Baseline - Majority Class: {testing_baseline_majority_class:.2f}")

<a id=Classification></a>

## Classification

<a id=setup_cls></a>

### Setup and Preprocessing

In [ ]:
py_cls = ClassificationExperiment()

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

setup_params_cls = {
    "log_experiment": True,
    "log_profile": False,
    "log_plots": False,
    "experiment_name": f"CLS_2_{timestamp}",
    "data": training_df,
    "test_data": testing_df,
    "target": "CLS_TARGET",
    "preprocess": False,
    "normalize": False,  # zscore
    "transformation": False,  # yeo-johnson power transform to make data more Gaussian
    "remove_outliers": False,  # using SVD
    "remove_multicollinearity": False,
    "polynomial_features": False,
    "feature_selection": False,
    "pca": False,
    "pca_components": 10,
    "numeric_features": [],
    "ignore_features": ["game_id"],
}

In [ ]:
py_cls.setup(**setup_params_cls)

<a id=compare_cls></a>

### Compare Models

In [ ]:
best_model_cls = py_cls.compare_models(turbo=True, sort="Accuracy", exclude=["catboost"])

In [ ]:
print(best_model_cls)

<a id=create_cls></a>

### Create Selected Model

In [ ]:
model_cls = py_cls.create_model("lr")

<a id=tune_cls></a>

### Tune Selected Model

In [ ]:
tuned_model_cls = py_cls.tune_model(model_cls)

In [ ]:
print(tuned_model_cls)

In [ ]:
model_report["details"] = tuned_model_cls.get_params()

<a id=evaluate_cls></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [ ]:
py_cls.evaluate_model(tuned_model_cls)

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [ ]:
# py_cls.interpret_model(tuned_model_cls)

In [ ]:
train_predictions_cls = py_cls.predict_model(tuned_model_cls, data=training_df)
train_prediction_metrics = py_cls.pull()

In [ ]:
model_report["train_accuracy"] = train_prediction_metrics["Accuracy"][0]
model_report["train_auc"] = train_prediction_metrics["AUC"][0]

In [ ]:
test_predictions_cls = py_cls.predict_model(tuned_model_cls, data=testing_df)
test_prediction_metrics = py_cls.pull()

In [ ]:
model_report["test_accuracy"] = test_prediction_metrics["Accuracy"][0]
model_report["test_auc"] = test_prediction_metrics["AUC"][0]

In [ ]:
roi_results_cls = calculate_roi(
    test_predictions_cls, "CLS_TARGET", "prediction_label", pred_prob="prediction_score"
)
roi_results_cls

In [ ]:
model_report["roi_all_bets_even_amount_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Even Amount"
]["Average ROI per Bet"].iloc[0]

model_report["roi_all_bets_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_50_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 50% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_50_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 50% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_55_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 55% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_55_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 55% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_60_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 60% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_60_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 60% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_65_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 65% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_65_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 65% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_70_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 70% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_70_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 70% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_all_bets_even_amount_kelly_criterion_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Even Amount, Kelly Criterion"
]["Average ROI per Bet"].iloc[0]

model_report["roi_all_bets_typical_odds_kelly_criterion_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Typical Odds, Kelly Criterion"
]["Average ROI per Bet"].iloc[0]

<a id=finalize_and_store_cls></a>

### Model Finalization and Storage

In [ ]:
final_model_cls = py_cls.finalize_model(tuned_model_cls)

In [ ]:
platform = "pycaret"
problem_type = "cls"
model_type = "lr"
datetime_str = model_report["datetime"].strftime("%Y_%m_%d_%H_%M_%S")

model_id = f"{platform}_{problem_type}_{model_type}_{datetime_str}"
model_id

In [ ]:
py_cls.save_model(final_model_cls, f"../models/AutoML/{model_id}")

In [ ]:
model_report["platform"] = platform
model_report["model_type"] = model_type
model_report["model_id"] = model_id

In [ ]:
model_report

In [ ]:
save_model_report(model_report)